In [ ]:
# runs voice activity detection and automatic speech recognition for
# all collected audio; requires manual steps in between

In [ ]:
import sys
sys.path.append('../python/')
from vad_asr import do_all, vad, asr
import db
import fio

In [ ]:
# run energy-based vad for all sessions of status 1 (logs done)
# (returns total sounding time)
db.connect()
do_all(vad)
db.commit()
db.close()
# after this, manually correct all VAD (and set status to 3)

In [ ]:
# check labels in manually corrected vad files of sessions of status 3
# (vad corrected), they have to be 'silent' & 'sounding', back and forth
db.connect()
for grp_id, ses_id, ses_type, rnd, mch_pair in db.find_sessions(3):
    for mch_id, wav_path, wav_fname in mch_pair:
        if mch_id == 0:
            # woz, no vad file
            continue
        vad_path, vad_fname = fio.get_vad_pfn(wav_fname, 'corrected/')
        print(vad_path + vad_fname)
        if not fio.check_vad_textgrid_file(vad_path, vad_fname):
            msg = 'vad file has errors; please fix them and rerun this'
            raise ValueError(msg)
db.close()

In [ ]:
# run asr for all sessions of status 3 (vad corrected)
db.connect()
do_all(asr)
db.commit()
db.close()
# after this, manually correct transcriptions (via psiturk)